In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [2]:
# Загружаем предварительно обученную модель VGG16
vgg16 = models.vgg16(pretrained=True)
vgg16.eval()  # переводим модель в режим оценки

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/andreisuhov/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [05:09<00:00, 1.79MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
# тестируем работоспособность нейронки, путем загрузки пробной фотки
image_path = "/Users/andreisuhov/Desktop/DL projects/нейроночки/dogdog.jpg"
input_image = Image.open(image_path)

# ресайзим до 256 на 256 пикселей, обрезаем, переводим в тензор и нормализирем, все как в документации vgg
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # Создайте пакет из одного изображения, чтобы была видимость бача для сетки

# используем GPU, если она доступна
if torch.cuda.is_available():
    vgg16.cuda()
    input_batch = input_batch.to('cuda')

#прогоняем изображение через сеть
with torch.no_grad():
    output = vgg16(input_batch)

# Получить предсказанный класс
_, predicted_class = torch.max(output, 1)

print(predicted_class.item())

162


## Заменить последний выходной слой

In [6]:
# нужное число выходных нейронов
OUTPUT_NEURONS = 5

# У вгг последние слои находятся в блоке classifier и он состоит из 6 слоев, мы обращаемся
# к этому блоку, к последнему (6) слою и забираем in_features, так как он понадобится для изменения 
# последнего слоя
num_of_in_features = vgg16.classifier[6].in_features

# заменяем число выходных нейронов своим числом
vgg16.classifier[6] = torch.nn.Linear(num_of_in_features, OUTPUT_NEURONS)


In [7]:
# смострим на получившуюся архитектуру
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1